# Atom feed of tweeted new preprints by subject area

Preprints published in the past seven days tweeted at least three times as atom feed, grouped by subject area.

In [1]:
import sys
sys.path.append('../')
!{sys.executable} -m pip install lxml
import ast
import pandas as pd
from datetime import datetime, date
from feedgen.feed import FeedGenerator
import lxml

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [2]:
date = str(date.today().strftime('%Y-%m-%d'))
subject_areas = ['covid', 'medical_health-sciences', 'bioinformatics', 'biochemistry_cell-biology', 'cancer', 'ecology', 'evolutionary-biology', 'genetics', 'microbiology', 'neurosciences', 'physiology', 'plant-biology', 'zoology','other-biological-sciences']
preprints = pd.read_csv('../preprint_tweets/1101/preprint_tweets_' + date + '.csv', index_col=0)

# filter by section, and group all COVID-related preprints together
subsets = {}
subsets['covid'] = preprints[preprints['covid'] == True]
print('COVID: ' + str(len(subsets['covid'])))
subsets['medical_health-sciences'] = preprints[((preprints['archive'] == 'medRxiv')  | (preprints['subject-area'] == 'Pathology') | (preprints['subject-area'] == 'Pharmacology and Toxicology'))  & (preprints['covid'] == False)]
print('Medical and Health Sciences: ' + str(len(subsets['medical_health-sciences'])))
subsets['bioinformatics'] = preprints[(preprints['subject-area'] == 'Bioinformatics') & (preprints['covid'] == False)]
print('Bioinformatics: ' + str(len(subsets['bioinformatics'])))
subsets['biochemistry_cell-biology'] = preprints[((preprints['subject-area'] == 'Biochemistry') | (preprints['subject-area'] == 'Cell Biology') | (preprints['subject-area'] == 'Developmental Biology') | (preprints['subject-area'] == 'Synthetic Biology') | (preprints['subject-area'] == 'Systems Biology') | (preprints['subject-area'] == 'Molecular Biology')) & (preprints['covid'] == False)]
print('Biochemistry and Cell Biology: ' + str(len(subsets['biochemistry_cell-biology'])))
subsets['ecology'] = preprints[(preprints['subject-area'] == 'Ecology') & (preprints['covid'] == False)]
subsets['cancer'] = preprints[((preprints['subject-area'] == 'Cancer Biology') | (preprints['subject-area'] == 'Hematology') | (preprints['subject-area'] == 'Oncology')) & (preprints['covid'] == False)]
print('Cancer: ' + str(len(subsets['cancer'])))
print('Ecology: ' + str(len(subsets['ecology'])))
subsets['evolutionary-biology'] = preprints[(preprints['subject-area'] == 'Evolutionary Biology') & (preprints['covid'] == False)]
print('Evolutionary Biology: ' + str(len(subsets['evolutionary-biology'])))
subsets['genetics'] = preprints[(preprints['subject-area'] == 'Genetics') | (preprints['subject-area'] == 'Genomics') & (preprints['covid'] == False)]
print('Genetics: ' + str(len(subsets['genetics'])))
subsets['microbiology'] = preprints[((preprints['subject-area'] == 'Microbiology') | (preprints['subject-area'] == 'Immunology')) & (preprints['covid'] == False)]
print('Microbiology: ' + str(len(subsets['microbiology'])))
subsets['neurosciences'] = preprints[(preprints['subject-area'] == 'Neuroscience') & (preprints['covid'] == False)]
print('Neurosciences: ' + str(len(subsets['neurosciences'])))
subsets['physiology'] = preprints[(preprints['subject-area'] == 'Physiology') | (preprints['subject-area'] == 'Biophysics') & (preprints['covid'] == False)]
print('Physiology: ' + str(len(subsets['physiology'])))
subsets['plant-biology'] = preprints[(preprints['subject-area'] == 'Plant Biology') & (preprints['covid'] == False)]
print('Plant Biology: ' + str(len(subsets['plant-biology'])))
subsets['zoology'] = preprints[(preprints['subject-area'] == 'Zoology') | (preprints['subject-area'] == 'Animal Behavior and Cognition') & (preprints['covid'] == False)]
print('Zoology: ' + str(len(subsets['zoology'])))
subsets['other-biological-sciences'] = preprints[(preprints['archive'] != 'medRxiv') & (preprints['subject-area'] != 'Animal Behavior and Cognition') & (preprints['subject-area'] != 'Bioinformatics') & (preprints['subject-area'] != 'Biochemistry') & (preprints['subject-area'] != 'Biophysics') & (preprints['subject-area'] != 'Cancer Biology') & (preprints['subject-area'] != 'Cell Biology') & (preprints['subject-area'] != 'Developmental Biology') & (preprints['subject-area'] != 'Ecology') & (preprints['subject-area'] != 'Evolutionary Biology') & (preprints['subject-area'] != 'Genetics') & (preprints['subject-area'] != 'Genomics') & (preprints['subject-area'] != 'Immunology') & (preprints['subject-area'] != 'Microbiology') & (preprints['subject-area'] != 'Molecular Biology') & (preprints['subject-area'] != 'Neuroscience') & (preprints['subject-area'] != 'Physiology') & (preprints['subject-area'] != 'Plant Biology') & (preprints['subject-area'] != 'Synthetic Biology') & (preprints['subject-area'] != 'Systems Biology') & (preprints['subject-area'] != 'Zoology') & (preprints['covid'] == False)]
print('Other Biological Sciences: ' + str(len(subsets['other-biological-sciences'])))
subsets['microbiology'].head(50)

COVID: 37
Medical and Health Sciences: 13
Bioinformatics: 17
Biochemistry and Cell Biology: 44
Cancer: 7
Ecology: 5
Evolutionary Biology: 14
Genetics: 11
Microbiology: 47
Neurosciences: 29
Physiology: 13
Plant Biology: 11
Zoology: 5
Other Biological Sciences: 9


,doi,tweets,archive,subject-area,covid,title,authors,abstract,posted
10,10.1101/2021.11.17.468999,79,bioRxiv,Microbiology,False,Reversible bacteriophage resistance by sheddin...,"[{'name': 'Veronique Ongenae'}, {'name': 'Adam...",<p>Phages are highly abundant in the environme...,2021-11-18
24,10.1101/2021.11.15.468770,29,bioRxiv,Microbiology,False,Comparative genomics on cultivated and unculti...,"[{'name': 'Hang Yu'}, {'name': 'Grayson L. Cha...",<p>Chemolithoautotrophic manganese oxidation h...,2021-11-16
39,10.1101/2021.11.15.468621,22,bioRxiv,Microbiology,False,Mobilome-driven segregation of the resistome i...,"[{'name': 'Laura de Nies'}, {'name': 'Susheel ...",<p>Biological wastewater treatment plants (BWW...,2021-11-15
44,10.1101/2021.11.16.467908,21,bioRxiv,Microbiology,False,Tuning spherical cells into kinking helices in...,"[{'name': 'Carole Lartigue'}, {'name': 'Bastie...","<p>In bacteria, cell shape is determined and m...",2021-11-17
54,10.1101/2021.11.17.468248,17,bioRxiv,Immunology,False,Broadly-neutralizing antibodies that bind to t...,"[{'name': 'Ali Zhang'}, {'name': 'Hanu Chaudha...",<p>The conserved hemagglutinin stalk domain is...,2021-11-19
62,10.1101/2021.11.15.468669,16,bioRxiv,Immunology,False,Spatially regulated protease activity in lymph...,"[{'name': 'Aereas Aung'}, {'name': 'Ang Cui'},...",<p>The structural integrity of vaccine antigen...,2021-11-15
66,10.1101/2021.11.18.469141,15,bioRxiv,Microbiology,False,Glacier-fed stream biofilms harbour diverse re...,"[{'name': 'Susheel Bhanu Busi'}, {'name': 'Lau...",<sec><title>Background</title><p>Antimicrobial...,2021-11-18
68,10.1101/2021.11.16.468763,14,bioRxiv,Microbiology,False,Disruption of vacuolin microdomains in the hos...,"[{'name': 'Cristina Bosmani'}, {'name': 'Angél...",<p><italic>Mycobacterium tuberculosis</italic>...,2021-11-16
69,10.1101/2021.11.17.468714,14,bioRxiv,Microbiology,False,NMR metabolomics of symbioses between bacteria...,"[{'name': 'Victoria Horrocks'}, {'name': 'Char...",<p>Bacterial vaginosis (BV) is a dysbiosis of ...,2021-11-17
71,10.1101/2021.11.18.469145,14,bioRxiv,Microbiology,False,Cryo soft X-ray tomography to explore Escheric...,"[{'name': 'Antoine Cossa'}, {'name': 'Sylvain ...",<p>Bacterial chromosomic DNA is packed within ...,2021-11-18


In [3]:

for subject_area in subject_areas:
    fg = FeedGenerator()
    fg.id('https://front-matter.io/')
    if subject_area == 'covid':
        fg.title(f'Popular BioRxiv/medRxiv COVID preprints posted in the last week')
    else:
        fg.title(f'Popular {subject_area.replace("-", " ").replace("_", " and ")} preprints posted in the last week')
    fg.author( {'name':'Martin Fenner','email':'martin@front-matter.io'} )
    fg.link( href='https://front-matter.io', rel='alternate' )
    fg.link( href=f'https://front-matter.io/{subject_area}/feed.xml', rel='self' )
    fg.language('en')

    for index, row in subsets[subject_area].iterrows():
        fe = fg.add_entry(order='append')
        fe.title(row['title'] + f" ({row['tweets']} tweets)")
        fe.author(ast.literal_eval(row['authors']))
        fe.id('https://doi.org/' + row['doi'])
        fe.link({'href': 'https://doi.org/' + row['doi'], 'title': row['title'] + f" ({row['tweets']} tweets)"})
        fe.description(row['abstract'])
        fe.category({ 'term': row['subject-area']})
        fe.published(datetime.fromisoformat(row['posted'] + 'T00:00:00+00:00'))
                 
    fg.atom_file(f'{subject_area}/feed.xml')